# Coursera Final Project

## Main Goal:

We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

## Resources: 

- https://www.kaggle.com/c/competitive-data-science-predict-future-sales
- https://www.kaggle.com/c/competitive-data-science-final-project
- https://www.kaggle.com/alessandrosolbiati/using-xgboost-for-time-series-prediction-top-20 
- https://www.kaggle.com/dimitreoliveira/model-stacking-feature-engineering-and-eda 
- https://www.coursera.org/learn/competitive-data-science/peer/QJDGf/final-project

## Feature description: 
- ID: an Id that represents a (Shop, Item) tuple within the test set
- shop_id: unique identifier of a shop
- item_id: unique identifier of a product
- item_category_id: unique identifier of item category
- item_cnt_day: number of products sold. You are predicting a monthly amount of this measure TARGET VALUE !!!
- item_price: current price of an item
- date: date in format dd/mm/yyyy
- date_block_num: a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
- item_name: name of item
- shop_name: name of shop
- item_category_name: name of item category

In [ ]:
############
#Libraries
############

import numpy as np
import pandas as pd 
from itertools import product
from tqdm import tqdm_notebook

import sklearn 
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import GridSearchCV
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Check the versions of the libraries
for p in [np, pd, sklearn, sns , xgb]:
    print (p.__name__, p.__version__)

In [ ]:
#############
# Datasets 
#############

items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv',parse_dates=True)#item info. item id and category id 
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv',parse_dates=True) # item_category_id
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv',parse_dates=True)# the name of the shops. shop_id
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=True)# training data 
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv',parse_dates=True) # testing data 
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv',parse_dates=True)

## 1 EDA

In [ ]:
# Add item information to train 

train = pd.merge(train, items, on='item_id', how='left')
train = train.drop('item_name', axis=1)
train.head()

In [ ]:
# Number of extra products on the train set compared to the train set  
set(train['shop_id'].unique()) - set(test['shop_id'].unique())

In [ ]:
# How many items and shop ids that we have on train we dont have on test?

train['shop_id'].unique()

In [ ]:
set(test['shop_id'].unique()) - set(train['shop_id'].unique()) 

In [ ]:
# How many uniques shops and items are in the data ?  

print(test['shop_id'].nunique()) # 42
print(train['shop_id'].nunique()) # 60
print(test['item_id'].nunique()) # 5100
print(train['item_id'].nunique()) # 21807

In [ ]:
# Target distribution 
sns.boxplot(x ='item_cnt_day' , data = train )
# There are some outliers

### Generating new features from text 

In [ ]:
# TEST 
# here we add sum , mean and count 
train_month = train.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_id'], as_index = False).agg({'item_price':['sum', 'mean', 'count'], 'item_cnt_day':['sum', 'mean','count'], 'item_category_id':['sum', 'mean', 'count']})
train_month.columns = ['date_block_num', 'shop_id', 'item_id', 'sum_item_price', 'mean_item_price', 'count_item_price', 'sum_item_cnt', 'mean_item_cnt', 'count_item_cnt', 'sum_item_sales', 'mean_item_sales', 'count_item_sales']
train_month.head()

In [ ]:
# Number of products sold 
train_month.groupby('date_block_num')['item_id'].count()

In [ ]:
train_month.groupby('date_block_num')['count_item_cnt'].sum()

In [ ]:
# what is the len here ? 
len(train_month)
# 1609124 I decrease the number here since I group by

In [ ]:
len(train)

In [ ]:
# TEST
# Grid with all combinations 
temp = []
for i in range(34):
    for shop in train_month['shop_id'].unique(): # shop uinique ids 
        for item in train_month['item_id'].unique(): #item unique ids 
            temp.append([i, shop, item]) # append to temp 
    
temp = pd.DataFrame(temp, columns=['date_block_num','shop_id','item_id']) # put temp on a dataframe 

train_month = pd.merge(temp, train_month, on = ['date_block_num','shop_id','item_id'], how = 'left')
train_month.fillna(0, inplace = True)

In [ ]:
block0 = train_month[train_month['date_block_num']==0]

block32 = train_month[train_month['date_block_num']== 32]

In [ ]:
block0['item_id'].value_counts()

In [ ]:
block32['item_id'].value_counts()

In [ ]:
train

In [ ]:
# TEST
# create month and year
train_month['year'] = train_month['date_block_num'].apply(lambda x: (x // 12 + 2013))
train_month['month'] = train_month['date_block_num'].apply(lambda x: (x % 12))

In [ ]:
len(train_month)

In [ ]:
train_month.groupby('month')['count_item_cnt'].sum().plot(figsize=(12,5))

#### Text
- Lowercase
- Lemmatization
- Stemming
- Stopwords

Pipeline: 
- 1 Preprocessing (lowercase, stemming, lemmatization or remove stopwords )
- 2 Ngrams can help to use local context 
- 3 Post processing using TFIdf


In [ ]:
# TEST
# 1 Create new features from text

# extract features from the text - transform item categories: 

# Translate the item category name and find meta category, subtype
symbols=(u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ", u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA")
english = {ord(a):ord(b) for a, b in zip(*symbols)}

# create 3 new rows from text 
item_categories['items_english'] = item_categories['item_category_name'].apply(lambda x: x.translate(english))
item_categories['meta_category'] = item_categories['items_english'].apply(lambda x: x.split('-')[0])
item_categories['subtype'] = item_categories['items_english'].apply(lambda x: x.split('-')) \
                                                             .map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
item_categories.head(10)

In [ ]:
# 2 Label encode these new features

# Apply label encoding to item categories
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
item_categories["meta_category_code"] = labelencoder.fit_transform(item_categories["meta_category"])
item_categories["subtype_code"] = labelencoder.fit_transform(item_categories["subtype"])
item_categories = item_categories.drop(["items_english", "meta_category", "subtype"], axis=1)

In [ ]:
item_categories.head()

In [ ]:
# Translate the shop name and find towns
shops["shops_english"] = shops["shop_name"].apply(lambda x: x.translate(english))
shops["town"] = shops["shops_english"].apply(lambda x: x.split()[0])
shops.head(5)

In [ ]:
# Label encoding
shops["town_code"] = labelencoder.fit_transform(shops["town"])
shops = shops.drop(["shops_english", "town"], axis=1)

In [ ]:
shops.head()

In [ ]:
# Make some useful features

# Remove dates

newcols = train["date"].str.split(".", expand=True)
ls = ["day", "month", "year"]
for i in range(len(ls)):
    train.insert(i, ls[i], newcols[i])
train['weekday'] = pd.to_datetime(train['date'], format = '%d.%m.%Y').dt.dayofweek
train.pop("date")

# Add item category for each item
train['item_category_id'] = [items['item_category_id'].values[i] for i in train['item_id'].values]

# Add revenues
train["revenue"] = train["item_price"]*train["item_cnt_day"]

In [ ]:
train.head()

In [ ]:
# Run this once 
train = train.merge(items, how='left')
train = train.merge(item_categories, how='left')
train = train.merge(shops, how='left')

train.drop("item_name",axis=1,inplace=True)
train.drop("shop_name",axis=1,inplace=True)
train.drop("item_category_name",axis=1,inplace=True)


In [ ]:
train.head()

## 2 Generate different combinations 

We put 0 where there are no item/store combination to facilitate the process for the machine learning model 

In [ ]:
#########
# Create index cols 
#########
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [ ]:
#########
# For every block num we generate unique combinations of shop_id and item_id 
#########

grid = list()

for block_num in train['date_block_num'].unique(): # 0-33
    """
    For every block num we generate unique combinations of shop_id and item_id 
    """
    cur_shops = train.loc[train['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = train.loc[train['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))# product will return cartesian product ex: # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy

grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32) # np.vstack will bring the array together 

In [ ]:
# temp = []
# for i in range(34):
#     for shop in train_month['shop_id'].unique(): # shop uinique ids 
#         for item in train_month['item_id'].unique(): #item unique ids 
#             temp.append([i, shop, item]) # append to temp 
    
# temp = pd.DataFrame(temp, columns=['date_block_num','shop_id','item_id']) # put temp on a dataframe 

# train_month = pd.merge(temp, train_month, on = ['date_block_num','shop_id','item_id'], how = 'left')
# train_month.fillna(0, inplace = True)

In [ ]:
grid.head() # here we have created all possible combinations for date_block_num 

In [ ]:
len(grid) # here we have created all possible combinations for date_block_num 

What is the diferecen of grid and the previous table ?
- Table 2 has len 10913850 while table 1 has 44486280
- 2935849 len of train 


In [ ]:
len(train)



## 3 Mean encoding
Here I added 9 new columns after item_category_id: 
- 'item_id_avg_item_price' 'item_id_sum_item_cnt_day' 'item_id_avg_item_cnt_day' 'shop_id_avg_item_price', 'shop_id_sum_item_cnt_day' 
'shop_id_avg_item_cnt_day' 'item_category_id_avg_item_price' 'item_category_id_sum_item_cnt_day' 'item_category_id_avg_item_cnt_day'

In [ ]:
#########
# Mean encoding -- create diferent combinations 
#########

mean_transactions = train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index() 

#After calculating the sum of the target variable and price_mean we merge it back to our grid and to items
mean_transactions = pd.merge(grid,mean_transactions,on=['date_block_num', 'shop_id', 'item_id'],how='left').fillna(0)
mean_transactions = pd.merge(mean_transactions, items, on='item_id',how='left')

In [ ]:
mean_transactions.head()

In [ ]:
len(mean_transactions)

In [ ]:
#########
# Mean encoding on difrent combinations to add 9 new feautures 
#########

# Now we apply mean encodings: 
for type_id in ['item_id', 'shop_id', 'item_category_id']:
    for column_id, aggregator, aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:
        
        mean_df = train.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id,'date_block_num']
        mean_transactions = pd.merge(mean_transactions, mean_df, on=['date_block_num',type_id], how='left')


In [ ]:
len(mean_transactions)

In [ ]:
print(f'mean transactions shape: {mean_transactions.shape}')
mean_transactions.head()


In [ ]:
# We still can see some outliers 
sns.boxplot(x ='item_cnt_day' , data = mean_transactions )

## 4 Generate lags

In [ ]:
#########
# Generate lags 
#########

# Create lags
lag_variables  = list(mean_transactions.columns[7:]) + ['item_cnt_day']
lags = [1, 2, 3, 6]
from tqdm import tqdm_notebook

for lag in tqdm_notebook(lags): 
    sales_new_df = mean_transactions.copy()
    sales_new_df.date_block_num += lag
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    mean_transactions = pd.merge(mean_transactions, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

In [ ]:
# Check all the columns
mean_transactions.columns

In [ ]:
#########
# Fill null values with zeros
#########

mean_transactions = mean_transactions[mean_transactions['date_block_num']>12] # from 13 to 33 

In [ ]:
# Null values were generated for lags 
mean_transactions.isnull().sum()

In [ ]:
# Sannity check 

# item_count = 0 
# item_price = 0 
# for feat in mean_transactions.columns:
#     if 'item_cnt' in feat:
#         item_count+= 1 
#     elif 'item_price' in feat:
#         item_price+= 1 

# item_count + item_price 

In [ ]:
#########
#  Fill null values with 0 and median 
#########

for feat in mean_transactions.columns:
    if 'item_cnt' in feat:
        mean_transactions[feat]=mean_transactions[feat].fillna(0)
    elif 'item_price' in feat:
        mean_transactions[feat]=mean_transactions[feat].fillna(mean_transactions[feat].median())

In [ ]:
#########
# Drop the columns that wont be able to have prediction time. This columns are not lagged 
#########

cols_to_drop = lag_variables[:-1] + ['item_price', 'item_name'] # dropping all target variables but not "item_cnt_day" cause is target
#mean_transactions[cols_to_drop].nunique()
training = mean_transactions.drop(cols_to_drop,axis=1)

In [ ]:
# Check the dataset
training.head()

In [ ]:
training.columns

## Modelling - Using XGBoost

In [ ]:
#########
#  Define the X and target:
#########

# Create a matrix and define X and y train 

#Grab  everything within the columns that are not the target 
X_train = training.iloc[:, training.columns != 'item_cnt_day'].values 

#Grab  everything within the column target 
y_train = training.iloc[:, training.columns == 'item_cnt_day'].values

xgbtrain = xgb.DMatrix(X_train , y_train)

Here we should try to do a grid search to find optimal paramaters. 

In [ ]:
#########
#  Define the hyperparamaters to pass to the model and train the model on these parameters (random paramaters)
#########

param = {'max_depth':10, 
         'subsample':1,
         'min_child_weight':0.5,
         'eta':0.3, 
         'num_round':1000, 
         'seed':1,
         'silent':0,
         'eval_metric':'rmse'} # random parameters

# Train the parameters on these metrics. Fit the model 

model1 = xgb.train(param, xgbtrain)


In [ ]:
#########
#  Plot the most important features 
#########

x = xgb.plot_importance(model1)
x.figure.set_size_inches(10, 30)

In [ ]:
cols = list(training.columns)
del cols[cols.index('item_cnt_day')] # eliminate target feature col name
[cols[x] for x in [2, 0, 5, 8, 4, 1, 3, 9, 33]]

## Preparing the test dataset
- We model the test set in the same way we modelled the train set. # So we need to manipulate the training set similarly to how we did in the first part of the notebook.

In [ ]:
#########
#  
#########

# here we give a value of 34 to all our block num  
test['date_block_num'] = 34 

In [ ]:
test.head()

In [ ]:
# Merge items to test set 
test = pd.merge(test, items, on='item_id', how='left')

In [ ]:
test.head()

In [ ]:
mean_transactions[['date_block_num','shop_id','item_id']]

In [ ]:
# cretae lags for test set 
#lags are lags = [1, 2, 3, 6]
for lag in tqdm_notebook(lags):

    sales_new_df = mean_transactions.copy()
    sales_new_df.date_block_num += lag
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables] # where did i compute this variables: lag_variables  = list(mean_transactions.columns[7:]) + ['item_cnt_day']
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    test = pd.merge(test, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

In [ ]:
test.head() 

In [ ]:
# Drop columns I wont use and check that test matches with train columns 
_test = set(test.drop(['ID', 'item_name'], axis=1).columns)
_training = set(training.drop('item_cnt_day',axis=1).columns)
for i in _test:
    assert i in _training
for i in _training:
    assert i in _test

In [ ]:
assert _training == _test

In [ ]:
# Drop the id and item_name
test = test.drop(['ID', 'item_name'], axis=1)

In [ ]:
for feat in test.columns:
    if 'item_cnt' in feat:
        test[feat]=test[feat].fillna(0)
    elif 'item_price' in feat:
        test[feat]=test[feat].fillna(test[feat].median())

In [ ]:
test[['shop_id','item_id']+['item_cnt_day_lag_'+str(x) for x in [1,2,3]]].head()

In [ ]:
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 33]['item_cnt_day'])
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 32]['item_cnt_day'])
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 31]['item_cnt_day'])

## Predict 

In [ ]:
# Create a xg matrix for test values 
xgbtest = xgb.DMatrix(test.values)

In [ ]:
# Predict on test set 

pred1 = model1.predict(xgbtest) # the model is model, the model is already fitted = xgb.train(param, xgbtrain)

In [ ]:
pred1 = pred1.clip(0, 20)

In [ ]:
final_submission = pd.DataFrame({'ID':test.index,'item_cnt_month': pred1 })

In [ ]:
# I cannot validate with no target on the test dataset 

final_submission.head()

In [ ]:
sample_submission.head()

In [ ]:
# To csv
#final_submission.to_csv('submissions.csv',index=False )

# Failed attempts

Reason :The kernel crashes, due to the RAM 

In [ ]:
# TEST
#model2 = XGBRegressor()

In [ ]:
#########
# 11.1 Hyperparam optimization
#########

# params = {'max_depth':[3,5,7,9,12,15,17,25], 
#          'subsample':[0.6,0.7,0.8,0.9,1],
#          'min_child_weight':[0.5,1,3,5,7],
#          'eta':[0.01,0.015,0.025,0.05, 0.1, 0.3],
#          'num_round':[1000], 
#          'seed':[1],
#          'silent':[0],
#          'eval_metric':['rmse']}


# grid_search = GridSearchCV(estimator= model2 ,param_grid= params, scoring= 'neg_root_mean_squared_error',cv=5 , n_jobs = -1 )

# grid_search.fit(X_train, y_train)

# grid_search.best_params_ 

## Solution 1 coursera

- He tested multiple regression models (lasso and regression) and the proceeded to random forest regressor  

## Solution 2 coursera